In [7]:
cd 'OneDrive - Intel Corporation\Documents\research_outside\geometricLWE\sage_notebook'

[WinError 3] The system cannot find the path specified: "'OneDrive - Intel Corporation\\Documents\\research_outside\\geometricLWE\\sage_notebook'"
c:\Users\huijingg\OneDrive - Intel Corporation\Documents\research_outside\geometricLWE\sage_notebook


In [13]:
load("../framework/instance_gen.sage")
demo = False
from scipy.io import loadmat
import random

"""  Example
Uncomment the following to get an example
of the detailed computation (without redundancy)
"""
demo = False
logging("--- Demonstration mode (no redundancy of the attacks) ---")


for params in ['CCS2']: #[ 'CCS2', 'CCS3', 'CCS4', 'NIST1', 'NIST2']:#'CCS1', 
    logging("Set of parameters: " + params)

    if params == 'NIST1':
        # NIST1 FRODOKEM-640
        n = 640
        m = 640
        q = 2**15
        frodo_distribution = [9456, 8857, 7280, 5249, 3321,
                              1844, 898, 384, 144, 47, 13, 3]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        # We used the following seeds for generating Bos et al. data
        # These seeds were generated with the matlab code genValues.m
        sca_seeds = [42, 72, 163, 175, 301, 320, 335, 406, 430, 445]
        param = 4

    elif params == 'NIST2':
        # NIST2 FRODOKEM-976
        n = 976
        m = 976
        q = 65536
        frodo_distribution = [11278, 10277, 7774, 4882, 2545, 1101,
                              396, 118, 29, 6, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [74, 324, 337, 425, 543, 1595, 1707, 2026, 2075, 2707]
        param = 5

    elif params == 'CCS1':
        n = 352
        m = 352
        q = 2 ** 11
        frodo_distribution = [22528, 15616, 5120, 768]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        param = 0

    elif params == 'CCS2':
        n = 592
        m = 592
        q = 2 ** 12
        frodo_distribution = [25120, 15840, 3968, 384, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [2, 3, 4, 5, 6, 7, 8, 9, 14, 16]
        param = 1


    elif params == 'CCS3':
        n = 752
        m = 752
        q = 2 ** 15
        frodo_distribution = [19296, 14704, 6496, 1664, 240, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [2, 4, 5, 7, 9, 10, 12, 13, 14, 15]
        param = 2

    elif params == 'CCS4':
        n = 864
        m = 864
        q = 2 ** 15
        frodo_distribution = [19304, 14700, 6490, 1659, 245, 21, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [256, 393, 509, 630, 637, 652, 665, 1202, 1264, 1387]
        param = 3


    """  Original Security   """

    A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                               n,
                                               q, m, D_s, D_s, verbosity=0)
    dbdd.integrate_q_vectors(q, indices=range(n, n + m))
    (beta, _) = dbdd.estimate_attack()
    logging("Attack without hints:  %3.2f bikz" % beta, style="HEADER")

    """  Refined Side channel attack  """

    # Reading the score tables from Bos et al. attack
    scores = []
    correct = []
    for seed in sca_seeds:
        for i in range(1, 9):
            data = loadmat('Scores_tables_SCA/' + params +
                           '/apostandcorrect' + str(param + 1) +
                           '_seed' + str(seed) +
                           'nbar' + str(i) + '.mat')
            scores += list(data['apostdist'])
            correct += list(data['correct'])
    measures = {}
    # the score tables are stored according to the secret coefficient. We use them
    # for generating the measurement.
    for key_guess in range(-len(frodo_distribution)+1, len(frodo_distribution)):
        measures[key_guess] = [scores[ind] for ind in
                  [i for i, d in enumerate(correct) if
                   recenter(d) == key_guess]]

    def simu_measured(secret):
        """
        This fonction simulates the information gained by
        Bos et al attack. For a given secret, the simulation
        is outputting a score table at random from the data of Bos et al.
        :secret: an integer being the secret value
        :measurement: a score table
        """
        if secret in measures.keys():
            measurement = random.choice(measures[secret])
        else: 
            measurement = None
        return measurement

    def measurement_to_aposteriori(measurement):
        """
        This fonction transforms a score table into an aposteriori distribution.
        According to the matlab code of Bos et al. attack,
        the score table is proportional
        to the logarithm of the aposteriori probability. We thus apply the exponential
        and renormalize.
        :measurement: a score table
        :apost_dist: a dictionnary of the aposteriori distribution
        """
        if measurement is not None:
            s = sum([exp(meas) for meas in measurement])
            measurement = [exp(meas)/s for meas in measurement]
            apost_dist = {}
            for key_guess in range(-len(frodo_distribution) + 1, len(frodo_distribution)):
                apost_dist[key_guess] = measurement[key_guess + len(frodo_distribution) - 1]
        else:
            apost_dist = None
        return apost_dist


    def estimate_SCA(report_every, dbdd, measured, max_guesses):
        """ 
        This function evaluates the security loss after Bos et al attack
        :report_every: an integer that give the frequency of
        logging (None for no logging)
        :dbdd: instance of the class DBDD
        :measured: table representing the (simulated) information
        given by Bos et al attack
        :max_guesses: integer for upperbounding the number of guesses
        """
        proba_best = {}
        coord_best1 = {} ###########################guess test
        Id = identity_matrix(n + m)
        for i in range(n):
            apost_dist = measurement_to_aposteriori(measured[i])
            if apost_dist is not None:
                proba_best[i] = max([apost_dist[j] for j in apost_dist.keys()])
                dist_j = [(j, apost_dist[j]) for j in apost_dist.keys()] ######################################### guess test
                coord_best1[i] = max(dist_j, key=lambda x:x[1])[0]  ######################################### guess test
                av, var = average_variance(apost_dist)
                av = float(av)
                var = float(var)
            else:
                av = None
                var = None
            v = vec(Id[i])
            if report_every is not None and ((i % report_every == 0) or (i == n - 1)) :
                verbose = 2 
            else:
                verbose = 0
            dbdd.verbosity = verbose
            if verbose == 2:
                logging("[...%d]" % report_every, newline=False)
            if var is not None and var > 0:
                dbdd.integrate_approx_hint(v, av, var,
                                           aposteriori=True, estimate=verbose)
            elif var is not None and var == 0 :
                logging("     Real Perfect Hint     ", style="HEADER")
                dbdd.integrate_perfect_hint(v, av, estimate=verbose)
        # if report_every is not None:
        #     dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m), report_every=report_every)
        # else:
        #     dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m))
        (beta, _) = dbdd.estimate_attack()
        print("beta after SCA: ", beta)
        # if report_every is not None:
        #     dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m), report_every=report_every)
        # else:
        #     dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m))
        (beta_after_q, _) = dbdd.estimate_attack()
        print("beta after SCA after q: ", beta_after_q)

        if report_every is not None:
            logging("     Hybrid attack estimation     ", style="HEADER")

        sorted_guesses = sorted(proba_best.items(),
                                key=lambda kv: - kv[1])
        sorted_guesses = [sorted_guesses[i][0] for i in range(len(sorted_guesses))
                          if sorted_guesses[i][1] != 1.]
        proba_success = 1.
        dbdd.verbosity = 0
        guesses = 0
        j = 0
        count_not_good_guess = 0
        last_included_prob = 0
        for i in sorted_guesses:
            j += 1
            if proba_success*proba_best[i] >= 0.77:#guesses <= 300: #max_guesses:#(proba_success >= max_guesses):
                v = vec(Id[i])
                if dbdd.integrate_perfect_hint(v, _, force = False):
                    guesses += 1
                    proba_success *= proba_best[i]
                    last_included_prob = proba_best[i]
                    # print(coord_best1[i], dbdd.u[0, i])
                    if coord_best1[i] != dbdd.u[0, i]:
                        count_not_good_guess += 1
                        print(guesses, "guess not good", coord_best1[i], dbdd.u[0, i], proba_best[i])
                # if report_every is not None and (j % report_every == 0):
                #     print("call q vector at the end")
                #     logging("[...%d]" % report_every, newline=False)
                #     dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m))
                #     logging("dim=%3d \t delta=%.6f \t beta=%3.2f \t guesses=%4d" %
                #             (dbdd.dim(), dbdd.delta, dbdd.beta, guesses),
                #             style="VALUE", newline=False)
                #     logging("Proba success = %s" % proba_success, style="VALUE",
                #             newline=True)
        print("count_not_good_guess = ", count_not_good_guess)
        print("last guess probability included", last_included_prob)
        beta_before_last_q = dbdd.beta
        print("beta_before_last_q", beta_before_last_q)
        if report_every is not None:
            dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m), report_every=report_every)
        else:
            dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m))
            
        return beta, dbdd.beta, proba_success, guesses

    if demo:
        A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                                  n,
                                                  q, m, D_s, D_s, verbosity=2)
        measured = [simu_measured(dbdd.u[0, i]) for i in range(n)]
        estimate_SCA(50, dbdd, measured, 0.5)
    else:
        """  Averaging
        The following averages the measures to get accurate data
        for the paper. The averaging mode is quite long.
        """
        nb_tests_per_params = 1

        # Chosen values for the number of guesses
        if params == 'CCS1':
            max_guesses = 175
        elif params == 'CCS2':
            max_guesses = 300
        elif params == 'CCS3':
            max_guesses = 250
        elif params == 'CCS4':
            max_guesses = 250
        elif params == 'NIST1':
            max_guesses = 100
        elif params == 'NIST2':
            max_guesses = 250

        beta = 0
        beta_hybrid = 0
        proba_success = 0
        guesses_average = 0
        for i in range(nb_tests_per_params):
            A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                                      n,
                                                      q, m, D_s, D_s, verbosity=0)
            measured = [simu_measured(dbdd.u[0, i]) for i in range(n)]
            b, b_hybrid, p_success, guesses = estimate_SCA(None, dbdd,
                                                  measured, max_guesses)
            beta += b
            beta_hybrid += b_hybrid
            proba_success += p_success
            guesses_average += guesses
            
        beta /= nb_tests_per_params
        beta_hybrid /= nb_tests_per_params
        proba_success /= nb_tests_per_params
        guesses_average /= nb_tests_per_params
        
        logging("Attack with hints: %3.2f bikz" % beta, style="HEADER")
        logging("Attack with hints and guess: %3.2f bikz"% beta_hybrid, style="HEADER")
        logging("Number of guesses: %4d" % guesses_average, style="HEADER")
        logging("Success probability: %3.2f" %proba_success, style="HEADER")



 --- Demonstration mode (no redundancy of the attacks) --- 
 Set of parameters: CCS2 
 Attack without hints:  447.85 bikz 
beta after SCA:  233.7156097163248
beta after SCA after q:  233.7156097163248
count_not_good_guess =  0
last guess probability included 0.9846193664797598
beta_before_last_q 133.73136432156255
 Attack with hints: 233.72 bikz 
 Attack with hints and guess: 91.66 bikz 
 Number of guesses:  374 
 Success probability: 0.77 


In [8]:
ls

 Volume in drive C is OSDisk
 Volume Serial Number is 3464-28EB

 Directory of c:\Users\huijingg\OneDrive - Intel Corporation\Documents\research_outside\geometricLWE\sage_notebook

05/12/2022  04:14 PM    <DIR>          .
05/12/2022  04:14 PM    <DIR>          ..
01/31/2022  11:55 AM    <DIR>          .vscode
12/08/2021  11:21 AM           416,516 compare_guess_intersection_order.ipynb
05/12/2022  04:12 PM                80 delete soon.txt
05/12/2022  04:12 PM               125 delete soon1.txt
05/12/2022  04:13 PM            14,706 delete_soon2.txt
05/12/2022  04:14 PM               911 delete_soon3.txt
05/11/2022  09:21 PM            21,654 exploiting_SCA_from_Bos_et_al.sage
05/12/2022  10:45 AM           313,616 generating_var_av.ipynb
05/02/2022  03:04 PM             8,688 optimize_r.sage
03/18/2022  11:30 AM           129,304 rerun_CCS1_to_CCS2.txt
04/19/2022  11:19 AM            33,388 rerun_CCS1_to_NIST2_by_5_samples.txt
03/15/2022  07:33 PM                 0 rerun_CCS3_to_NIST2

In [2]:
load("../framework/instance_gen.sage")
demo = False
from scipy.io import loadmat
import random

"""  Example
Uncomment the following to get an example
of the detailed computation (without redundancy)
"""
# demo = True
# logging("--- Demonstration mode (no redundancy of the attacks) ---")

# import time
# start = time.time()
# print("time count starts...", start)
# start_one_parameter = start

for params in ['NIST2']: #'CCS1', 'CCS2', 'CCS3', 'CCS4', 'NIST1', 
    logging("Set of parameters: " + params)

    if params == 'NIST1':
        # NIST1 FRODOKEM-640
        n = 640
        m = 640
        q = 2**15
        frodo_distribution = [9456, 8857, 7280, 5249, 3321,
                              1844, 898, 384, 144, 47, 13, 3]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        # We used the following seeds for generating Bos et al. data
        # These seeds were generated with the matlab code genValues.m
        sca_seeds = [42, 72, 163, 175, 301, 320, 335, 406, 430, 445]
        param = 4

    elif params == 'NIST2':
        # NIST2 FRODOKEM-976
        n = 976
        m = 976
        q = 65536
        frodo_distribution = [11278, 10277, 7774, 4882, 2545, 1101,
                              396, 118, 29, 6, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [74, 324, 337, 425, 543, 1595, 1707, 2026, 2075, 2707]
        param = 5

    elif params == 'CCS1':
        n = 352
        m = 352
        q = 2 ** 11
        frodo_distribution = [22528, 15616, 5120, 768]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        param = 0

    elif params == 'CCS2':
        n = 592
        m = 592
        q = 2 ** 12
        frodo_distribution = [25120, 15840, 3968, 384, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [2, 3, 4, 5, 6, 7, 8, 9, 14, 16]
        param = 1


    elif params == 'CCS3':
        n = 752
        m = 752
        q = 2 ** 15
        frodo_distribution = [19296, 14704, 6496, 1664, 240, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [2, 4, 5, 7, 9, 10, 12, 13, 14, 15]
        param = 2

    elif params == 'CCS4':
        n = 864
        m = 864
        q = 2 ** 15
        frodo_distribution = [19304, 14700, 6490, 1659, 245, 21, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [256, 393, 509, 630, 637, 652, 665, 1202, 1264, 1387]
        param = 3


    """  Original Security   """

    A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                               n,
                                               q, m, D_s, D_s, verbosity=0)
    dbdd.integrate_q_vectors(q, indices=range(n, n + m))
    (beta, _) = dbdd.estimate_attack()
    logging("Attack without hints:  %3.2f bikz" % beta, style="HEADER")

    """  Refined Side channel attack  """

    # Reading the score tables from Bos et al. attack
    scores = []
    correct = []
    for seed in sca_seeds:
        for i in range(1, 9):
            data = loadmat('Scores_tables_SCA/' + params +
                           '/apostandcorrect' + str(param + 1) +
                           '_seed' + str(seed) +
                           'nbar' + str(i) + '.mat')
            scores += list(data['apostdist'])
            correct += list(data['correct'])
    measures = {}
    # the score tables are stored according to the secret coefficient. We use them
    # for generating the measurement.
    for key_guess in range(-len(frodo_distribution)+1, len(frodo_distribution)):
        measures[key_guess] = [scores[ind] for ind in
                  [i for i, d in enumerate(correct) if
                   recenter(d) == key_guess]]

    def simu_measured(secret):
        """
        This fonction simulates the information gained by
        Bos et al attack. For a given secret, the simulation
        is outputting a score table at random from the data of Bos et al.
        :secret: an integer being the secret value
        :measurement: a score table
        """
        if secret in measures.keys():
            measurement = random.choice(measures[secret])
        else: 
            measurement = None
        return measurement

    def measurement_to_aposteriori(measurement):
        """
        This fonction transforms a score table into an aposteriori distribution.
        According to the matlab code of Bos et al. attack,
        the score table is proportional
        to the logarithm of the aposteriori probability. We thus apply the exponential
        and renormalize.
        :measurement: a score table
        :apost_dist: a dictionnary of the aposteriori distribution
        """
        if measurement is not None:
            s = sum([exp(meas) for meas in measurement])
            measurement = [exp(meas)/s for meas in measurement]
            apost_dist = {}
            for key_guess in range(-len(frodo_distribution) + 1, len(frodo_distribution)):
                apost_dist[key_guess] = measurement[key_guess + len(frodo_distribution) - 1]
        else:
            apost_dist = None
        return apost_dist


    def estimate_SCA(report_every, dbdd, measured, max_guesses):
        """ 
        This function evaluates the security loss after Bos et al attack
        :report_every: an integer that give the frequency of
        logging (None for no logging)
        :dbdd: instance of the class DBDD
        :measured: table representing the (simulated) information
        given by Bos et al attack
        :max_guesses: integer for upperbounding the number of guesses
        """
        proba_best = {}
        Id = identity_matrix(n + m)
        for i in range(n):
            apost_dist = measurement_to_aposteriori(measured[i])
            if apost_dist is not None:
                proba_best[i] = max([apost_dist[j] for j in apost_dist.keys()])
                av, var = average_variance(apost_dist)
                av = float(av)
                var = float(var)
            else:
                av = None
                var = None
            v = vec(Id[i])
            if report_every is not None and ((i % report_every == 0) or (i == n - 1)) :
                verbose = 2 
            else:
                verbose = 0
            dbdd.verbosity = verbose
            if verbose == 2:
                logging("[...%d]" % report_every, newline=False)
            if var is not None and var > 0:
                dbdd.integrate_approx_hint(v, av, var,
                                           aposteriori=True, estimate=verbose)
            elif var is not None and var == 0 :
                dbdd.integrate_perfect_hint(v, av, estimate=verbose)
        # if report_every is not None:
        #    dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m), report_every=report_every)
        # else:
        #     dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m))
        (beta, _) = dbdd.estimate_attack()

        if report_every is not None:
            logging("     Hybrid attack estimation     ", style="HEADER")

        sorted_guesses = sorted(proba_best.items(),
                                key=lambda kv: - kv[1])
        sorted_guesses = [sorted_guesses[i][0] for i in range(len(sorted_guesses))
                          if sorted_guesses[i][1] != 1.]
        proba_success = 1.
        dbdd.verbosity = 0
        guesses = 0
        j = 0
        for i in sorted_guesses:
            j += 1
            if (guesses <= max_guesses):
                v = vec(Id[i])
                if dbdd.integrate_perfect_hint(v, _, force = False):
                    guesses += 1
                    proba_success *= proba_best[i]
                if report_every is not None and (j % report_every == 0):
                    logging("[...%d]" % report_every, newline=False)
                    dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m))
                    logging("dim=%3d \t delta=%.6f \t beta=%3.2f \t guesses=%4d" %
                            (dbdd.dim(), dbdd.delta, dbdd.beta, guesses),
                            style="VALUE", newline=False)
                    logging("Proba success = %s" % proba_success, style="VALUE",
                            newline=True)

        if report_every is not None:
           dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m), report_every=report_every)
        else:
            dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m))

        return beta, dbdd.beta, proba_success

    if demo:
        A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                                  n,
                                                  q, m, D_s, D_s, verbosity=2)
        measured = [simu_measured(dbdd.u[0, i]) for i in range(n)]
        estimate_SCA(50, dbdd, measured, 500)
    else:
        """  Averaging
        The following averages the measures to get accurate data
        for the paper. The averaging mode is quite long.
        """
        nb_tests_per_params = 50

        # Chosen values for the number of guesses
        if params == 'CCS1':
            max_guesses = 175
        elif params == 'CCS2':
            max_guesses = 300
        elif params == 'CCS3':
            max_guesses = 250
        elif params == 'CCS4':
            max_guesses = 250
        elif params == 'NIST1':
            max_guesses = 100
        elif params == 'NIST2':
            max_guesses = 250

        beta = 0
        beta_hybrid = 0
        proba_success = 0
        for i in range(nb_tests_per_params):
            print("sample", i)
            A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                                      n,
                                                      q, m, D_s, D_s, verbosity=0)
            measured = [simu_measured(dbdd.u[0, i]) for i in range(n)]
            b, b_hybrid, p_success = estimate_SCA(None, dbdd,
                                                  measured, max_guesses)
            beta += b
            beta_hybrid += b_hybrid
            proba_success += p_success
            print("sample computation complete", i)
            print("beta", beta)
            print("beta_hybrid", beta_hybrid)
            print("proba_success", proba_success)

        beta /= nb_tests_per_params
        beta_hybrid /= nb_tests_per_params
        proba_success /= nb_tests_per_params
        
        logging("Attack with hints: %3.2f bikz" % beta, style="HEADER")
        logging("Attack with hints and guess: %3.2f bikz"% beta_hybrid, style="HEADER")
        logging("Number of guesses: %4d" % max_guesses, style="HEADER")
        logging("Success probability: %3.2f" %proba_success, style="HEADER")
        # end_one_parameter = time.time()
        # print(params, "takes time with 50 sample", end_one_parameter - start_one_parameter)
        # start_one_parameter = end_one_parameter

# end = time.time()
# print("time count ends...", end)
# print("total time for all parameters with 1 sample", end - start)
# Below is the old estimation code that did not directly 
# handle the score table by transforming it into an
# a posteriori distribution.

# load("../framework/instance_gen.sage")
# demo = False

# """  Example
# Uncomment the following to get an example
# of the detailed computation (without redundancy)
# """
# # demo = True
# # logging("--- Demonstration mode (no averaging) ---")


# for params in ['CCS1', 'CCS2', 'CCS3', 'CCS4', 'NIST1', 'NIST2']:
#     logging("Set of parameters: " + params)

#     if params == 'NIST1':
#         # NIST1 FRODOKEM-640
#         n = 640
#         m = 640
#         q = 2**15
#         frodo_distribution = [9288, 8720, 7216, 5264, 3384,
#                               1918, 958, 422, 164, 56, 17, 4, 1]
#         D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
#         load("Frodo_Single_data/simulation_distribution_NIST1.sage")
#         load("Frodo_Single_data/aposteriori_distribution_NIST1.sage")

#     elif params == 'NIST2':
#         # NIST2 FRODOKEM-976
#         n = 976
#         m = 976
#         q = 65536
#         frodo_distribution = [11278, 10277, 7774, 4882, 2545, 1101,
#                               396, 118, 29, 6, 1]
#         D_s = get_distribution_from_table(frodo_distribution, 2 ** 16) 
#         load("Frodo_Single_data/simulation_distribution_NIST2.sage")
#         load("Frodo_Single_data/aposteriori_distribution_NIST2.sage")

#     elif params == 'CCS1':
#         n = 352
#         m = 352
#         q = 2 ** 11
#         frodo_distribution = [22528, 15616, 5120, 768]
#         D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
#         load("Frodo_Single_data/simulation_distribution_CCS1.sage")
#         load("Frodo_Single_data/aposteriori_distribution_CCS1.sage")

#     elif params == 'CCS2':
#         n = 592
#         m = 592
#         q = 2 ** 12
#         frodo_distribution = [25120, 15840, 3968, 384, 16]
#         D_s = get_distribution_from_table(frodo_distribution, 2 ** 16) 
#         load("Frodo_Single_data/simulation_distribution_CCS2.sage")
#         load("Frodo_Single_data/aposteriori_distribution_CCS2.sage")

#     elif params == 'CCS3':
#         n = 752
#         m = 752
#         q = 2 ** 15
#         frodo_distribution = [19296, 14704, 6496, 1664, 240, 16]
#         D_s = get_distribution_from_table(frodo_distribution, 2 ** 16) 
#         load("Frodo_Single_data/simulation_distribution_CCS3.sage")
#         load("Frodo_Single_data/aposteriori_distribution_CCS3.sage")

#     elif params == 'CCS4':
#         n = 864
#         m = 864
#         q = 2 ** 15
#         frodo_distribution = [19304, 14700, 6490, 1659, 245, 21, 1]
#         D_s = get_distribution_from_table(frodo_distribution, 2 ** 16) 
#         load("Frodo_Single_data/simulation_distribution_CCS4.sage")
#         load("Frodo_Single_data/aposteriori_distribution_CCS4.sage")


#     """  Original Security   """

#     A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
#                                                n,
#                                                q, m, D_s, D_s, verbosity=0)
#     dbdd.integrate_q_vectors(q, indices=range(n, n + m))
#     (beta, _) = dbdd.estimate_attack()
#     logging("Attack without hints:  %3.2f bikz" % beta, style="HEADER")

#     """  Refined Side channel attack  """

#     def simu_measured(secret):
#         """
#         This fonction simulates the information gained by
#         Bos et al attack. The simulation is based on a
#         distribution obtained with a large amount of data
#         for Bos et al suite (in Matlab).
#         :secret: an integer being the secret value
#         :measurement: an integer that represents the output
#         of Bos et al attack.
#         """
#         secret = recenter(secret)
#         distrib_of_guesses = renormalize_dist(Dguess[secret])
#         measurement = draw_from_distribution(distrib_of_guesses)
#         return measurement


#     def ordered_indices(sorted_guesses, measured):
#             """
#             Necessary for the bruteforce attack, this function
#             sorts the indices of the coefficients
#             of the secret with decreasing likelihood.
#             :sorted_guess: the best guesses in order of likelihood
#             :measured: the measurement for each coefficient
#             :orderered_coefficients: the indices of the coefficients
#             ordered according to Probability[secret[i] = measured[i]]
#             """
#             orderered_coefficients = []
#             for x in sorted_guesses:
#                 for i in range(len(measured)):
#                     meas = measured[i]
#                     if meas == x:
#                         orderered_coefficients += [i]
#             return orderered_coefficients


#     def estimate_SCA(report_every, dbdd, measured, max_guesses):
#         """ 
#         This function evaluates the security loss after Bos et al attack
#         :report_every: an integer that give the frequency of
#         logging (None for no logging)
#         :dbdd: instance of the class DBDD
#         :measured: table representing the (simulated) information
#         given by Bos et al attack
#         :max_guesses: integer for upperbounding the number of guesses
#         """

#         Id = identity_matrix(n + m)
#         for i in range(n):
#             v = vec(Id[i])
#             if report_every is not None and ((i % report_every == 0) or (i == n - 1)) :
#                 verbose = 2 
#             else:
#                 verbose = 0
#             dbdd.verbosity = verbose
#             if verbose == 2:
#                 logging("[...%d]" % report_every, newline=False)
#             if variance_aposteriori[measured[i]] is not None and variance_aposteriori[measured[i]] != 0:
#                 dbdd.integrate_approx_hint(v,
#                                            center_aposteriori[measured[i]],
#                                            variance_aposteriori[measured[i]],
#                                            aposteriori=True, estimate=verbose)
#             elif variance_aposteriori[measured[i]] is not None and variance_aposteriori[measured[i]] == 0 :
#                 dbdd.integrate_perfect_hint(v, center_aposteriori[measured[i]],
#                                             estimate=verbose)
#         if report_every is not None:
#             dbdd.integrate_q_vectors(q, indices=range(n, n + m), report_every=report_every)
#         else:
#             dbdd.integrate_q_vectors(q, indices=range(n, n + m))
#         (beta, _) = dbdd.estimate_attack()

#         if report_every is not None:
#             logging("     Hybrid attack estimation     ", style="HEADER")

#         sorted_guesses = sorted(proba_best_guess_correct.items(),
#                                 key=lambda kv: - kv[1])
#         sorted_guesses = [sorted_guesses[i][0] for i in range(len(sorted_guesses))
#                           if sorted_guesses[i][1] != 1.]
#         proba_success = 1.
#         dbdd.verbosity = 0
#         guesses = 0
#         j = 0
#         for i in ordered_indices(sorted_guesses, measured):
#             j += 1
#             if (guesses <= max_guesses):
#                 v = vec(Id[i])
#                 if dbdd.integrate_perfect_hint(v, _):
#                     guesses += 1
#                     proba_success *= proba_best_guess_correct[measured[i]]
#                 if report_every is not None and (j % report_every == 0):
#                     logging("[...%d]" % report_every, newline=False)
#                     dbdd.integrate_q_vectors(q, indices=range(n, n + m))
#                     logging("dim=%3d \t delta=%.6f \t beta=%3.2f \t guesses=%4d" %
#                             (dbdd.dim(), dbdd.delta, dbdd.beta, guesses),
#                             style="VALUE", newline=False)
#                     logging("Proba success = %s" % proba_success, style="VALUE",
#                             newline=True)
#         return beta, dbdd.beta, proba_success

#     if demo:
#         A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
#                                                   n,
#                                                   q, m, D_s, D_s, verbosity=2)
#         measured = [simu_measured(dbdd.u[0, i]) for i in range(n)]
#         estimate_SCA(50, dbdd, measured, 200)
#     else:
#         """  Averaging
#         The following averages the measures to get accurate data
#         for the paper. The averaging mode is quite long.
#         """
#         nb_tests_per_params = 100

#         # Chosen values for the number of guesses
#         if params == 'CCS1':
#             max_guesses = 100
#         elif params == 'CCS2':
#             max_guesses = 350
#         elif params == 'CCS3':
#             max_guesses = 300
#         elif params == 'CCS4':
#             max_guesses = 150
#         elif params == 'NIST1':
#             max_guesses = 50
#         elif params == 'NIST2':
#             max_guesses = 100

#         beta = 0
#         beta_hybrid = 0
#         proba_success = 0
#         for i in range(nb_tests_per_params):
#             A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
#                                                       n,
#                                                       q, m, D_s, D_s, verbosity=0)
#             measured = [simu_measured(dbdd.u[0, i]) for i in range(n)]
#             b, b_hybrid, p_success = estimate_SCA(None, dbdd,
#                                                   measured, max_guesses)
#             beta += b
#             beta_hybrid += b_hybrid
#             proba_success += p_success

#         beta /= nb_tests_per_params
#         beta_hybrid /= nb_tests_per_params
#         proba_success /= nb_tests_per_params
#         logging("Attack with hints: %3.2f bikz" % beta, style="HEADER")
#         logging("Attack with hints and guess: %3.2f bikz"% beta_hybrid, style="HEADER")
#         logging("Number of guesses: %4d" % max_guesses, style="HEADER")
#         logging("Success probability: %3.2f" %proba_success, style="HEADER")

 Set of parameters: NIST2 
 Attack without hints:  708.06 bikz 
sample 0
sample computation complete 0
beta 481.2300917838392
beta_hybrid 338.8633499552077
proba_success 0.879890385828768
sample 1
sample computation complete 1
beta 950.7129532368599
beta_hybrid 657.5340806505656
proba_success 1.68661971304444
sample 2
sample computation complete 2
beta 1416.3232011511536
beta_hybrid 991.3966682609416
proba_success 2.58347051151100
sample 3
sample computation complete 3
beta 1887.8174320148637
beta_hybrid 1321.8270820984635
proba_success 3.47418779548851
sample 4
sample computation complete 4
beta 2372.4634625831945
beta_hybrid 1665.682633117524
proba_success 4.32730209097433
sample 5
sample computation complete 5
beta 2853.6063217280807
beta_hybrid 2004.4829433895236
proba_success 5.19032809303411
sample 6
sample computation complete 6
beta 3333.3934641974383
beta_hybrid 2346.3683880699955
proba_success 6.05945735683311
sample 7
sample computation complete 7
beta 3792.4482491598455
bet

In [3]:
load("../framework/instance_gen.sage")
demo = False
from scipy.io import loadmat
import random

"""  Example
Uncomment the following to get an example
of the detailed computation (without redundancy)
"""
# demo = True
# logging("--- Demonstration mode (no redundancy of the attacks) ---")

# import time
# start = time.time()
# print("time count starts...", start)
# start_one_parameter = start

for params in ['CCS2', 'CCS3', 'CCS4']: #'CCS1', 'CCS2', 'CCS3', 'CCS4', 'NIST1','NIST2' 
    logging("Set of parameters: " + params)

    if params == 'NIST1':
        # NIST1 FRODOKEM-640
        n = 640
        m = 640
        q = 2**15
        frodo_distribution = [9456, 8857, 7280, 5249, 3321,
                              1844, 898, 384, 144, 47, 13, 3]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        # We used the following seeds for generating Bos et al. data
        # These seeds were generated with the matlab code genValues.m
        sca_seeds = [42, 72, 163, 175, 301, 320, 335, 406, 430, 445]
        param = 4

    elif params == 'NIST2':
        # NIST2 FRODOKEM-976
        n = 976
        m = 976
        q = 65536
        frodo_distribution = [11278, 10277, 7774, 4882, 2545, 1101,
                              396, 118, 29, 6, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [74, 324, 337, 425, 543, 1595, 1707, 2026, 2075, 2707]
        param = 5

    elif params == 'CCS1':
        n = 352
        m = 352
        q = 2 ** 11
        frodo_distribution = [22528, 15616, 5120, 768]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        param = 0

    elif params == 'CCS2':
        n = 592
        m = 592
        q = 2 ** 12
        frodo_distribution = [25120, 15840, 3968, 384, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [2, 3, 4, 5, 6, 7, 8, 9, 14, 16]
        param = 1


    elif params == 'CCS3':
        n = 752
        m = 752
        q = 2 ** 15
        frodo_distribution = [19296, 14704, 6496, 1664, 240, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [2, 4, 5, 7, 9, 10, 12, 13, 14, 15]
        param = 2

    elif params == 'CCS4':
        n = 864
        m = 864
        q = 2 ** 15
        frodo_distribution = [19304, 14700, 6490, 1659, 245, 21, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [256, 393, 509, 630, 637, 652, 665, 1202, 1264, 1387]
        param = 3


    """  Original Security   """

    A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                               n,
                                               q, m, D_s, D_s, verbosity=0)
    dbdd.integrate_q_vectors(q, indices=range(n, n + m))
    (beta, _) = dbdd.estimate_attack()
    logging("Attack without hints:  %3.2f bikz" % beta, style="HEADER")

    """  Refined Side channel attack  """

    # Reading the score tables from Bos et al. attack
    scores = []
    correct = []
    for seed in sca_seeds:
        for i in range(1, 9):
            data = loadmat('Scores_tables_SCA/' + params +
                           '/apostandcorrect' + str(param + 1) +
                           '_seed' + str(seed) +
                           'nbar' + str(i) + '.mat')
            scores += list(data['apostdist'])
            correct += list(data['correct'])
    measures = {}
    # the score tables are stored according to the secret coefficient. We use them
    # for generating the measurement.
    for key_guess in range(-len(frodo_distribution)+1, len(frodo_distribution)):
        measures[key_guess] = [scores[ind] for ind in
                  [i for i, d in enumerate(correct) if
                   recenter(d) == key_guess]]

    def simu_measured(secret):
        """
        This fonction simulates the information gained by
        Bos et al attack. For a given secret, the simulation
        is outputting a score table at random from the data of Bos et al.
        :secret: an integer being the secret value
        :measurement: a score table
        """
        if secret in measures.keys():
            measurement = random.choice(measures[secret])
        else: 
            measurement = None
        return measurement

    def measurement_to_aposteriori(measurement):
        """
        This fonction transforms a score table into an aposteriori distribution.
        According to the matlab code of Bos et al. attack,
        the score table is proportional
        to the logarithm of the aposteriori probability. We thus apply the exponential
        and renormalize.
        :measurement: a score table
        :apost_dist: a dictionnary of the aposteriori distribution
        """
        if measurement is not None:
            s = sum([exp(meas) for meas in measurement])
            measurement = [exp(meas)/s for meas in measurement]
            apost_dist = {}
            for key_guess in range(-len(frodo_distribution) + 1, len(frodo_distribution)):
                apost_dist[key_guess] = measurement[key_guess + len(frodo_distribution) - 1]
        else:
            apost_dist = None
        return apost_dist


    def estimate_SCA(report_every, dbdd, measured, max_guesses):
        """ 
        This function evaluates the security loss after Bos et al attack
        :report_every: an integer that give the frequency of
        logging (None for no logging)
        :dbdd: instance of the class DBDD
        :measured: table representing the (simulated) information
        given by Bos et al attack
        :max_guesses: integer for upperbounding the number of guesses
        """
        proba_best = {}
        Id = identity_matrix(n + m)
        for i in range(n):
            apost_dist = measurement_to_aposteriori(measured[i])
            if apost_dist is not None:
                proba_best[i] = max([apost_dist[j] for j in apost_dist.keys()])
                av, var = average_variance(apost_dist)
                av = float(av)
                var = float(var)
            else:
                av = None
                var = None
            v = vec(Id[i])
            if report_every is not None and ((i % report_every == 0) or (i == n - 1)) :
                verbose = 2 
            else:
                verbose = 0
            dbdd.verbosity = verbose
            if verbose == 2:
                logging("[...%d]" % report_every, newline=False)
            if var is not None and var > 0:
                dbdd.integrate_approx_hint(v, av, var,
                                           aposteriori=True, estimate=verbose)
            elif var is not None and var == 0 :
                dbdd.integrate_perfect_hint(v, av, estimate=verbose)
        # if report_every is not None:
        #    dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m), report_every=report_every)
        # else:
        #     dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m))
        (beta, _) = dbdd.estimate_attack()

        if report_every is not None:
            logging("     Hybrid attack estimation     ", style="HEADER")

        sorted_guesses = sorted(proba_best.items(),
                                key=lambda kv: - kv[1])
        sorted_guesses = [sorted_guesses[i][0] for i in range(len(sorted_guesses))
                          if sorted_guesses[i][1] != 1.]
        proba_success = 1.
        dbdd.verbosity = 0
        guesses = 0
        j = 0
        for i in sorted_guesses:
            j += 1
            if (proba_success * proba_best[i] >= max_guesses): #(guesses <= max_guesses):
                v = vec(Id[i])
                if dbdd.integrate_perfect_hint(v, _, force = False):
                    guesses += 1
                    proba_success *= proba_best[i]
                if report_every is not None and (j % report_every == 0):
                    logging("[...%d]" % report_every, newline=False)
                    dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m))
                    logging("dim=%3d \t delta=%.6f \t beta=%3.2f \t guesses=%4d" %
                            (dbdd.dim(), dbdd.delta, dbdd.beta, guesses),
                            style="VALUE", newline=False)
                    logging("Proba success = %s" % proba_success, style="VALUE",
                            newline=True)

        if report_every is not None:
           dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m), report_every=report_every)
        else:
            dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m))

        return beta, dbdd.beta, proba_success, guesses

    if demo:
        A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                                  n,
                                                  q, m, D_s, D_s, verbosity=2)
        measured = [simu_measured(dbdd.u[0, i]) for i in range(n)]
        estimate_SCA(50, dbdd, measured, 500)
    else:
        """  Averaging
        The following averages the measures to get accurate data
        for the paper. The averaging mode is quite long.
        """
        nb_tests_per_params = 20

        # Chosen values for the number of guesses
        # if params == 'CCS1':
        #     max_guesses = 175
        # elif params == 'CCS2':
        #     max_guesses = 300
        # elif params == 'CCS3':
        #     max_guesses = 250
        # elif params == 'CCS4':
        #     max_guesses = 250
        # elif params == 'NIST1':
        #     max_guesses = 100
        # elif params == 'NIST2':
        #     max_guesses = 250

        if params == 'CCS1':
            max_guesses = 0.86
        elif params == 'CCS2':
            max_guesses = 0.64
        elif params == 'CCS3':
            max_guesses = 0.87
        elif params == 'CCS4':
            max_guesses = 0.77
        elif params == 'NIST1':
            max_guesses = 0.81
        elif params == 'NIST2':
            max_guesses = 0.84

        beta = 0
        beta_hybrid = 0
        proba_success = 0
        guesses_ave = 0
        for i in range(nb_tests_per_params):
            print("sample", i)
            A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                                      n,
                                                      q, m, D_s, D_s, verbosity=0)
            measured = [simu_measured(dbdd.u[0, i]) for i in range(n)]
            b, b_hybrid, p_success, guesses = estimate_SCA(None, dbdd,
                                                  measured, max_guesses)
            beta += b
            beta_hybrid += b_hybrid
            proba_success += p_success
            guesses_ave += guesses
            
            print("beta", beta)
            print("beta_hybrid", beta_hybrid)
            print("proba_success", proba_success)
            print("number of gueeses: ", guesses_ave)
            print("sample computation complete", i)

        beta /= nb_tests_per_params
        beta_hybrid /= nb_tests_per_params
        proba_success /= nb_tests_per_params
        guesses_ave /= nb_tests_per_params
        
        logging("Attack with hints: %3.2f bikz" % beta, style="HEADER")
        logging("Attack with hints and guess: %3.2f bikz"% beta_hybrid, style="HEADER")
        logging("Number of guesses: %4d" % guesses_ave, style="HEADER")
        logging("Success probability: %3.2f" %proba_success, style="HEADER")

        # end_one_parameter = time.time()
        # print(params, "takes time with 50 sample", end_one_parameter - start_one_parameter)
        # start_one_parameter = end_one_parameter

# end = time.time()
# print("time count ends...", end)
# print("total time for all parameters with 1 sample", end - start)


 Set of parameters: CCS2 
 Attack without hints:  447.85 bikz 
sample 0


KeyboardInterrupt: 

In [12]:
print(beta/33, beta_hybrid/33, proba_success/33)
mention we are working on the extension, during the work on some extension, we discover this.  
We noticed the q vector issue which effect the result of beta.  I want to get the confirmationl


227.1520720623421 111.31652332516558 0.965720386152189


In [1]:
beta_record = beta
beta_hybrid_record = beta_hybrid
proba_success_record = proba_success
num_record = 35

NameError: name 'beta' is not defined

In [ ]:
sample 16
sample computation complete 16
beta 3959.6251671679656
beta_hybrid 1486.6598980972287
proba_success 11.0011696116790
sample 17

In [1]:
11.0011696116790/17

0.647127624216412

In [2]:
1486.6598980972287/17

87.45058224101345